In [2]:
%load_ext autoreload
%autoreload 2

from datasets import load_from_disk
import pickle
import torch
from torch import utils
import pytorch_lightning as pl

from nli.data import DataSetPadding
from nli.models import Baseline, MLP
from nli.learner import NLINet

In [3]:
with open('store/wordvec.pkl', 'rb') as f:
    wordvec = pickle.load(f)
dataset_snli = load_from_disk('data/snli')

In [4]:
dataloader = {}
for split, shuffle in zip(['train', 'validation'], [True, False]):
    dataset = DataSetPadding(dataset_snli[split], wordvec)
    if split == 'train':
        dataset = utils.data.Subset(dataset, range(10000))
    dataloader[split] = utils.data.DataLoader(dataset, batch_size=64, shuffle=shuffle, num_workers=16)

In [7]:
class UniLSTM(torch.nn.Module):
    def __init__(self):
        super(UniLSTM, self).__init__()
        self.lstm = torch.nn.LSTM(input_size = 300, hidden_size = 2048, 
                                  num_layers = 1, dropout=0, bidirectional=False, 
                                  batch_first=True)

    def forward(self, embedding, length):


        x = torch.nn.utils.rnn.pack_padded_sequence(embedding, length, batch_first=True, enforce_sorted=False)
        x, _ = self.lstm(x)


        # thisis wrong, should update dis
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(x, batch_first=True, total_length=100)

        print('x', x.shape)

        return x


encoder = UniLSTM()
# classifier = MLP(300*4, 512, 3)
classifier = MLP(2048*4, 512, 3)
model = NLINet(encoder, classifier)

device = 'gpu' if torch.cuda.is_available() else 'cpu'
trainer = pl.Trainer(max_epochs=10, log_every_n_steps=1, accelerator=device, enable_checkpointing=False)
trainer.fit(model, train_dataloaders = dataloader['train'], val_dataloaders = dataloader['validation'])

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name       | Type             | Params
------------------------------------------------
0 | encoder    | UniLSTM          | 19.3 M
1 | classifier | MLP              | 4.5 M 
2 | loss_fn    | CrossEntropyLoss | 0     
------------------------------------------------
23.7 M    Trainable params
0         Non-trainable params
23.7 M    Total params
94.841    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

x torch.Size([64, 100, 2048])
x torch.Size([64, 100, 2048])
u, v torch.Size([64, 100, 2048]) torch.Size([64, 100, 2048])
features torch.Size([64, 400, 2048])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (25600x2048 and 8192x512)